In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv(r"D:\DataSets\nn_train.csv")
df_test = pd.read_csv(r"D:\DataSets\nn_test.csv")
df_train = df_train.drop(columns=['ID'])
df_train


In [ ]:
def shuffle(df_arr):
    np.random.seed(0)
    np.random.shuffle(df_arr)
    np.random.seed(None)
    return

df_train = np.array(df_train)
shuffle(df_train)
df_train[:5]

In [ ]:
X_train = df_train[:,:1024]
binary_label = df_train[:,1024]
class_label = df_train[:,1025]

print(f"Shape of Tarining set: {X_train.shape}")
print(f"Shape of Binary Label: {binary_label.shape}")
print(f"Shape of class label: {class_label.shape}")
print(f"Unique values in Binary label is: {np.unique(binary_label)}")
print(f"Unique values in Class label is: {np.unique(class_label)}")
print(f"Maximum value of data is: {np.max(X_train)}")
print(f"Minimum value of data is: {np.min(X_train)}")

n_unique, n_counts_train = np.unique(binary_label, return_counts=True)

print(f"Total number of 1 and 0 in Training  set:\n1: {n_counts_train[1]}\n0: {n_counts_train[0]}")

In [ ]:
def normalize(x):
    # x_mean = x.mean()
    # x_std = x.std()
    # x_normal = (x-x_mean)/x_std
    return x/255

X_train_normalize = normalize(X_train)
X_train_normalize

In [33]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [34]:
def softmax(z):
    z_max = np.max(z, axis=1, keepdims=True)
    exp_z = np.exp(z - z_max)
    return exp_z/(np.sum(exp_z, axis=1, keepdims=True))

In [35]:
def sigmoid_derivative(z):
    return sigmoid(z)*(1 - sigmoid(z))

In [36]:
def cross_entropy_loss(y_true, y_pred):
    y_pred = np.where(y_pred<=1e-12, 1e-12, y_pred)
    loss = np.sum(y_true*np.log(y_pred), axis=1)
    cost = -np.mean(loss)
    return cost

In [ ]:
def one_hot_encode(labels):
    labels = labels.astype(int)
    n_labels = len(labels)
    n_unique_labels= len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    for i in range(n_labels):
        one_hot_encode[i,labels[i]] = 1
    return one_hot_encode

binary_one_hot = one_hot_encode(binary_label)

class_label = class_label-1
class_one_hot = one_hot_encode(class_label)
class_label = class_label + 1
class_label

In [45]:
n = len(X_train[0])
m = len(X_train)

a1 = 512
a2 = 256
a3 = 128
a4 = 64
binary_op = 2
class_op = 10

w1 = np.random.rand(n, a1)*np.sqrt(1/n)
w2 = np.random.rand(a1, a2)*np.sqrt(1/a1)
w3 = np.random.rand(a2, a3)*np.sqrt(1/a2)
w4 = np.random.rand(a3, a4)*np.sqrt(1/a3)
w_binary = np.random.rand(a4, binary_op)*np.sqrt(1/a4)
w_class = np.random.rand(binary_op, class_op)*np.sqrt(1/a4)

b1 = np.zeros((m, a1))
b2 = np.zeros((m, a2))
b3 = np.zeros((m, a3))
b4 = np.zeros((m, a4))
b_binary = np.zeros((m, binary_op))
b_class = np.zeros((m, class_op))


In [ ]:
epoch = 100
learning_rate = 0.01
batch_size = 1000

for i in range(epoch):
    for j in range(0, m, batch_size):
        batch_x = X_train_normalize[j:j+batch_size]
        binary_one_hot_batch = binary_one_hot[j:j+batch_size]
        class_one_hot_batch = class_one_hot[j:j+batch_size]
        binary_Y = binary_label[j:j+batch_size]
        class_Y = class_label[j:j+batch_size]

        #forward propogation
        z1 = np.dot(batch_x, w1) + b1[j:j+batch_size]
        a1 = sigmoid(z1)
        z2 = np.dot(a1, w2) + b2[j:j+batch_size]
        a2 = sigmoid(z2)
        z3 = np.dot(a2, w3) + b3[j:j+batch_size]
        a3 = sigmoid(z3)
        z4 = np.dot(a3, w4) + b4[j:j+batch_size]
        a4 = sigmoid(z4)
        z5 = np.dot(a4, w_binary) + b_binary[j:j+batch_size]
        binary_op = softmax(z5)
        z6 = np.dot(binary_op, w_class) + b_class[j:j+batch_size]
        class_op  = softmax(z6)
       
        #backword propogation
        class_err = class_op - class_one_hot_batch
        binary_err = class_err.dot(w_class.T)*sigmoid_derivative(binary_op)
        err_a4 = binary_err.dot(w_binary.T)*sigmoid_derivative(a4)
        err_a3 = err_a4.dot(w4.T)*sigmoid_derivative(a3)
        err_a2 = err_a3.dot(w3.T)*sigmoid_derivative(a2)
        err_a1 = err_a2.dot(w2.T)*sigmoid_derivative(a1)

        #weight updation
        w_class -= learning_rate*(binary_op.T).dot(class_err)
        w_binary -= learning_rate*(a4.T).dot(binary_err)
        w4 -= learning_rate*(a3.T).dot(err_a4)
        w3 -= learning_rate*(a2.T).dot(err_a3)
        w2 -= learning_rate*(a1.T).dot(err_a2)
        w1 -= learning_rate*(batch_x.T).dot(err_a1)
        
        #biaes updation
        b_class[j:j+batch_size] -= learning_rate*class_err
        b_binary[j:j+batch_size] -= learning_rate*binary_err
        b4[j:j+batch_size] -= learning_rate*err_a4
        b3[j:j+batch_size] -= learning_rate*err_a3
        b2[j:j+batch_size] -= learning_rate*err_a2
        b1[j:j+batch_size] -= learning_rate*err_a1

    binary_loss = cross_entropy_loss(binary_one_hot_batch, binary_op)
    class_loss = cross_entropy_loss(class_one_hot_batch, class_op)
    binary_prediction = np.argmax(binary_op, axis=1)
    class_prediction = np.argmax(class_op, axis=1)
    binary_accuracy = np.mean((binary_prediction+1)==binary_Y)*100
    class_accuracy = np.mean(class_prediction == class_Y)*100
    print(f"Epoch: {1+i} Binary Loss: {binary_loss:.3f} Binary_Accuracy: {binary_accuracy:.3f} Class_loss: {class_loss:.3f} Class_Accuracy: {class_accuracy}")



In [ ]:
a1 = sigmoid(np.dot(df_test,w1) + b1[:len(df_test)])
a2 = sigmoid(np.dot(a1, w2) + b2[:len(df_test)])
a3 = softmax(np.dot(a2, w3) + b3[:len(df_test)])
a4 = sigmoid(np.dot(a3, w4) + b4[:len(df_test)])
binary_test_prediction = softmax(np.dot(a4, w_binary) + b_binary[:len(df_test)])
class_test_prediction = softmax(np.dot(binary_prediction, w_class) + b_class[:len(df_test)])

binary_prediction = np.argmax(binary_test_prediction, axis=1)
class_prediction = np.argmax(class_test_prediction, axis=1)

print(f"Binary Prediction: {binary_test_prediction}")
print(f"Class Prediction: {class_test_prediction}")
